In [2]:
!pip install nltk

  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 1.5MB 432kB/s ta 0:00:011
  Running setup.py bdist_wheel for nltk ... done
  Stored in directory: /home/chinmay/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk


In [1]:
#installing all the libraries needed for the task
import pandas as pd
import json
import nltk
from pandas.io.json import json_normalize
nltk.download('stopwords')
nltk.download('punkt')
import re  
from nltk.corpus import stopwords
stops1 = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/chinmay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/chinmay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
! pip install gensim
! pip install seaborn
! pip install elasticsearch
! pip install esengine
! pip install ipdb

In [2]:
# %cd /home/chinmay/Desktop/TUM/Sem-4/GuidedResearch/MyAttempts/github-code-guided-research/
from Vocabulary import Vocab

In [3]:
%cd /home/chinmay/Desktop/TUM/Sem-4/GuidedResearch/MyAttempts/

/home/chinmay/Desktop/TUM/Sem-4/GuidedResearch/MyAttempts


In [4]:
from preprocess import *
from sub_find import *
from true_subs import *
from constants import *
# from graph_evaluation import *
# from Vocabulary import Vocab
import pickle

In [5]:
from recipe import Recipe
def load_data_from_file(filename=None, ing_file_name=None, local=True, save=False):
    return load_data(file_name=filename, ing_file_name=ing_file_name, local=local, save=save)
    

In [6]:
import os
os.chdir('raw_data')
recipes = load_data_from_file("100k/100ksample", ing_file_name = "dbpedia_ingredients.txt",local=True, save=False )
clean_recipes = [x.proccessed_ing_list for x in recipes]
recipes = load_data_from_file("100k/100ksample2", "dbpedia_ingredients.txt")
sample100k2 = [x.proccessed_ing_list for x in recipes]
clean_recipes.extend(sample100k2) 
type(clean_recipes)

list

In [7]:
# currently looking only on the second set of elements
total_number_of_recipes = len(recipes)
print(total_number_of_recipes)

100000


In [8]:
titles_present = set()
for reduced_recipe_object in recipes:
  titles_present.add(str(reduced_recipe_object.id))

In [9]:
vars(recipes[0])

{'id': 'Cajun Spice Blend',
 'title': 'Cajun Spice Blend',
 'instructions_text': 'mix together and store in airtight container and use as wanted.',
 'ingredients_text': '1 tablespoon garlic powder 1 tablespoon onion powder 1 tablespoon sugar 1 teaspoon salt 1 teaspoon pepper 1 teaspoon dried basil 1 teaspoon paprika 12 teaspoon cayenne pepper',
 'proccessed_ing_list': ['garlic powder',
  'onion powder',
  'sugar',
  'cayenne pepper']}

In [10]:
final_dataset = [(x.proccessed_ing_list, x.id) for x in recipes]
final_dataset[0]

(['garlic powder', 'onion powder', 'sugar', 'cayenne pepper'],
 'Cajun Spice Blend')

In [11]:
# let us use nltk to perform stemming. That way, it is possible to get the matches better
# eg muffins and muffin

In [12]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
# type(titles_present)

In [13]:
def stem_title(word):
    return stemmer.stem(word)


In [14]:
# an example
stem_title('Carsons Chicken Wellington W/ Exotic Mushroom Sherry Sauce')

'carsons chicken wellington w/ exotic mushroom sherry sauc'

In [15]:
updated_title = [stem_title(title) for title in titles_present]

In [16]:
with open('recipe_titles_stemmed.txt', 'w') as file:
    for item in updated_title:
        file.write("%s\n" % item)

In [17]:
my_list = 'cake,pasta,custard,rolls,stew,casserole,sauce,soup,burgers,pizza,salad,muffins,steaks,fish,ham'.split(",")

In [18]:
vindhya_list = ['Cake', 'Salad', 'Noodles', 'Hummus', 'Burger', 'Steak', 'Pie', 'Custard', 'Chips', 'Casserole', 'Pasta',
                'Pizza', 'Omelette', 'Soup', 'Sauce','Sandwich','Muffin','Pickles','Cocktail','Pudding','Wraps','Taco','Daal',
                'Guacamole','Doughnut','Bread','Smoothie','Sorbet','Chili','Stew','Chicken','Beef',]

In [19]:
vindhya_map = {
    "Stew" : ["Daal", "Chili", "Goulash", "Stew"],
    "Cake" :["Muffin", "Cake"],
    "Condiments" : ["Pickles", "Sauce"],
    "Taco" : ["Quessidila", "Fajitas", "Taco"],
    "Salad" : ["Guacamole", "Salad"],
    "Noodles" : ["Noodles"],
    'Hummus' : ['Hummus'],
    'Burger' : ['Burger'],
    'Steak' : ['Steak'],
    'Pie' : ['Pie'],
    'Custard' : ['Custard'],
    'Chips' : ['Chips'],
    'Casserole' : ['Casserole'],
    'Pasta' : ['Pasta'],
    'Pizza' : ['Pizza'],
    'Omelette' : ['Omelette'],
    'Soup' : ['Soup'],
    'Sandwich' : ['Sandwich'],
    'Cocktail' : ['Cocktail'],
    'Pudding' : ['Pudding'],
    'Wraps' : ['Wraps'],
    'Doughnut' : ['Doughnut'],
    'Bread' : ['Bread'],
    'Smoothie' : ['Smoothie'],
    'Sorbet' : ['Sorbet'],
    'Chicken' : ['Chicken'],
    'Beef' : ['Beef'],
    
}

In [55]:
vindhya_map = dict((stemmer.stem(k), list(map(stemmer.stem, v))) for k, v in vindhya_map.items())

In [20]:
import itertools
all_recipe_list = list(itertools.chain(*vindhya_map.values()))

In [21]:
# check if the recipe matches
# https://stackoverflow.com/questions/6531482/how-to-check-if-a-string-contains-an-element-from-a-list-in-python
counter = 0
list(itertools.chain(*vindhya_map.values()))
for reduced_recipe_object in recipes:
    # Honestly this is a shitty attempt since I already did the stemming earlier
    # however, I am doing the same again
    counter = counter +  any (stem_title(title) in stem_title(reduced_recipe_object.id) for title in all_recipe_list)
    
counter

42053

In [22]:
percentage_covered = counter / total_number_of_recipes * 100
print(percentage_covered)

42.053000000000004


# Eventually we will be using the cleaned recipe list for our evaluation

In [23]:
# so sample 100k2
final_dataset = [(x.proccessed_ing_list, x.id) for x in recipes]
only_ingredients = [i[0] for i in final_dataset]

In [24]:
type(final_dataset[0])

tuple

## DONE : Fix the code here. I just used glove pretrined so things can be ignored for a while here

In [25]:
ingredient_vocabulary = Vocab()
for recipe in only_ingredients:
    for ingredients in recipe:
        ingredient_vocabulary.add_word(ingredients)

In [26]:
# Let us see the vocabulary size once
ingredient_vocabulary.max_idx 

1152

In [27]:
ingredient_vocabulary.get_word(100)

'hamburger'

In [28]:
import pandas as pd
df = pd.DataFrame(final_dataset, columns =['Ingredients', 'Recipe_id'])

In [29]:
df.iloc[5]

Ingredients    [orange, clove, apple, water, cinnamon]
Recipe_id                     The Spiced Cider Project
Name: 5, dtype: object

## Word2Vec was huge hence glove 100d  file is used ~200MB

#### Use this block only for the first time since it simply does the conversion of the Embeddings. Also, in future, it makes more sense to upload this converted file into colab while running the code

In [30]:
# ! wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
# ! gzip -d GoogleNews-vectors-negative300.bin.gz
from gensim.test.utils import datapath
from gensim.scripts.glove2word2vec import glove2word2vec
glove_file = datapath('/home/chinmay/Desktop/TUM/Sem-4/GuidedResearch/dataset/glove/glove.6B.100d.txt')
with open('/home/chinmay/Desktop/TUM/Sem-4/GuidedResearch/dataset/glove_w2vformat/glove.100d.w2v', mode='wb+') as dest_file:
    glove2word2vec(glove_file, dest_file)


In [31]:
import gensim
pretrained_embedding = gensim.models.KeyedVectors.load_word2vec_format('~/Desktop/TUM/Sem-4/GuidedResearch/dataset/glove_w2vformat/glove.100d.w2v')

In [32]:
# Just a sample check to indicate things are working as expected
pretrained_embedding.vocab['orange'].index

3200

In [33]:
# This is more of a hack
def recipe_vocab_on_list(ingedient_list):
    result_list = []
    for ingedient in ingedient_list:
        # If multi word ingredient, chose the first word
        ingedient = ingedient.split(" ")[0]
        try:
            result_list.append(pretrained_embedding.vocab[ingedient].index)
        except KeyError:
            result_list.append(pretrained_embedding.vocab['unk'].index)
    return result_list
    

In [34]:
df['Ingredient_Numeric'] = df.Ingredients.apply(recipe_vocab_on_list)

In [35]:
df.iloc[5]

Ingredients           [orange, clove, apple, water, cinnamon]
Recipe_id                            The Spiced Cider Project
Ingredient_Numeric            [3200, 33170, 3292, 430, 17773]
Name: 5, dtype: object

In [36]:
recipe_vocabulary = Vocab()
for recipe in vindhya_map.keys():
    recipe_vocabulary.add_word(stemmer.stem(recipe))

In [37]:
recipe_vocabulary

{'<UNK>': 0, 'stew': 1, 'cake': 2, 'condiment': 3, 'taco': 4, 'salad': 5, 'noodl': 6, 'hummu': 7, 'burger': 8, 'steak': 9, 'pie': 10, 'custard': 11, 'chip': 12, 'casserol': 13, 'pasta': 14, 'pizza': 15, 'omelett': 16, 'soup': 17, 'sandwich': 18, 'cocktail': 19, 'pud': 20, 'wrap': 21, 'doughnut': 22, 'bread': 23, 'smoothi': 24, 'sorbet': 25, 'chicken': 26, 'beef': 27}

In [38]:
len(recipe_vocabulary.word_2_idx)

28

In [57]:
recipe_vocabulary.get_all_words_in_vocab_mit_map(stemmer.stem('The Spiced Cake Project'),
                                                vindhya_map)
# recipe_vocabulary.get_idx('cake')

(True, [2])

In [58]:
# Uptil this point we were simply returning the first match in the recipe vocabulary. 
# This can be modified and we look for all the matches of possible recipe
# Since the list is sorted specific to our rule, we would go ahead and take the minimum value from all
def get_recipe_idx(recipe):
    recipe_stemmed = stemmer.stem(recipe)
    lookup_result = recipe_vocabulary.get_all_words_in_vocab_mit_map(recipe_stemmed, vindhya_map)
    if lookup_result[0]:
        print(lookup_result[1])
        return min(lookup_result[1])
    # No match so 0 returned
    return  0

In [59]:
get_recipe_idx('The Spiced Cake Project')

[2]


2

In [60]:
df['Recipe_id_numeric'] = df.Recipe_id.apply(get_recipe_idx)

[19]
[27, 1]
[26, 5]
[26]
[5]
[26, 13]
[13]
[23]
[26]
[26]
[15]
[5]
[17]
[26]
[18]
[3]
[12]
[14]
[3]
[26]
[26, 9]
[26]
[26, 17]
[3]
[26]
[5]
[19]
[26, 13]
[4, 17]
[26]
[23]
[17]
[10]
[15]
[2]
[17]
[1]
[3]
[2]
[17]
[27]
[26]
[15]
[1]
[18]
[10]
[12]
[23]
[1]
[19]
[23]
[26, 17]
[26]
[27]
[2]
[27]
[2]
[23]
[5]
[5]
[13]
[2]
[2]
[5]
[19]
[26]
[14, 26]
[26, 17]
[6]
[24]
[13]
[26]
[26]
[26, 3]
[10]
[1]
[26, 18]
[5]
[17]
[6]
[26]
[14]
[1]
[5]
[2, 3]
[26, 5]
[5]
[5]
[10]
[26, 5]
[5]
[13]
[5]
[26]
[13]
[26]
[15]
[26]
[24]
[14, 3]
[27]
[17]
[26]
[27, 17]
[26]
[5, 18]
[19]
[17]
[27]
[12]
[3]
[3]
[14]
[26]
[10]
[27]
[23]
[1]
[17]
[5]
[3]
[17]
[5]
[26]
[27, 18]
[23]
[17]
[26]
[9]
[27]
[10]
[26]
[23]
[2]
[3]
[12, 2]
[3]
[5]
[5]
[22]
[3]
[17]
[3]
[17]
[26]
[5]
[26]
[15]
[26]
[5]
[15]
[17]
[17]
[3]
[3]
[23]
[3]
[3]
[26]
[24]
[13]
[23]
[27, 13]
[15]
[26]
[3]
[17]
[23]
[23]
[23]
[26]
[5]
[2]
[26]
[3]
[3]
[26]
[2]
[26, 3]
[26]
[5]
[17]
[4]
[26]
[14]
[3]
[3]
[26, 17]
[10]
[27]
[26]
[26]
[2]
[18]
[26]
[14]
[

[27]
[26]
[14, 26]
[5]
[17]
[5]
[26]
[26]
[17]
[17]
[12]
[20]
[12]
[26]
[26]
[23]
[26]
[26]
[14]
[2]
[5]
[9]
[5]
[26]
[26]
[9]
[2]
[26]
[3]
[27, 5]
[2, 3]
[3]
[3]
[5]
[23]
[4, 14, 5]
[23]
[26]
[26]
[26]
[13]
[13]
[5]
[12]
[26, 5]
[17]
[17]
[13]
[9]
[14]
[5]
[17]
[5]
[2]
[23]
[17]
[26]
[5]
[20]
[26, 13]
[1]
[5]
[27, 18]
[27, 9]
[24]
[24]
[17]
[23]
[14]
[3]
[3]
[26]
[2]
[19]
[2]
[23]
[5]
[23]
[26]
[15]
[26]
[26]
[23]
[13]
[23]
[26]
[5]
[27]
[26]
[26, 1, 3]
[17]
[17]
[26]
[19]
[23]
[15]
[18]
[23]
[5]
[1, 23]
[5]
[17]
[5]
[4]
[26, 17]
[18]
[26, 5]
[18]
[1]
[3]
[14]
[23]
[27]
[26]
[26]
[23]
[5]
[9, 3]
[14]
[24]
[17]
[26]
[25]
[26]
[17]
[14]
[5]
[13]
[10]
[1, 26]
[17]
[23]
[26]
[23]
[12]
[17]
[2]
[11, 23]
[5]
[12]
[17]
[23]
[5]
[5]
[5]
[1]
[9]
[24]
[5]
[3]
[19]
[13]
[26, 4]
[23]
[27]
[17]
[26, 17]
[3]
[27, 5]
[3]
[17]
[26]
[17, 2]
[15]
[5]
[26]
[26, 5]
[17]
[1]
[5]
[16]
[12, 2]
[3]
[20]
[5]
[17]
[5]
[3]
[27]
[26]
[13]
[26]
[3]
[27, 18]
[23]
[10]
[19]
[3]
[26]
[18]
[2]
[9]
[23]
[20]
[3]
[3]
[

[15]
[26, 26]
[3]
[26]
[13]
[26, 17]
[26]
[12]
[3]
[2]
[26]
[9, 3]
[2]
[5]
[10]
[9]
[26, 3]
[12]
[3]
[18]
[3]
[26]
[26]
[26, 5]
[15]
[2]
[21]
[22]
[5]
[5]
[26]
[3]
[26, 1]
[26]
[17]
[26]
[17]
[7]
[26]
[24]
[26]
[17]
[5, 18]
[17]
[23]
[10, 20]
[26]
[10]
[26]
[26]
[1, 3]
[10]
[3]
[3]
[10]
[20]
[17]
[3]
[26, 18]
[26]
[26, 9]
[17]
[24]
[26]
[26, 14]
[10]
[27, 6]
[3]
[26]
[26]
[2]
[23]
[26]
[15]
[5]
[5]
[26]
[1, 3]
[26, 3]
[26, 3]
[27]
[18]
[26]
[5]
[26, 5]
[26, 3]
[26]
[1]
[5]
[10]
[5]
[2]
[5]
[26, 14]
[5]
[27, 13]
[13]
[14]
[17]
[3]
[17]
[4]
[17]
[5]
[6]
[26]
[26, 23]
[26]
[26]
[17]
[26]
[26]
[2]
[11]
[3]
[26, 3]
[17]
[26]
[17]
[5]
[3]
[26, 15]
[26]
[27]
[2]
[20]
[23]
[1]
[5]
[2]
[4, 17]
[17]
[20, 2]
[23]
[26, 18]
[9]
[5]
[26]
[26]
[24]
[5]
[17]
[17]
[26]
[4]
[5, 1]
[17]
[27, 17]
[26]
[17]
[26]
[9]
[23]
[26]
[3]
[10]
[9]
[18]
[27]
[12]
[1]
[5]
[5]
[5]
[26]
[3]
[26]
[2]
[5]
[5]
[1]
[26, 21]
[1]
[15]
[12, 2]
[5]
[9]
[17]
[4]
[5]
[5]
[3]
[17]
[5, 27]
[26, 17]
[11]
[4]
[4]
[5]
[2]
[14, 5]
[2,

[2]
[23]
[27, 1]
[26]
[14]
[17]
[23]
[26, 1]
[23]
[1]
[5]
[18]
[12]
[26]
[27]
[23]
[1]
[26]
[13]
[27]
[26]
[2]
[26, 5]
[27]
[14]
[5]
[26, 3]
[23, 20]
[1]
[9, 21]
[26]
[3]
[17]
[26]
[3]
[12]
[10]
[14]
[20]
[5]
[26]
[17]
[23]
[17]
[13]
[23]
[19]
[17]
[26]
[14]
[5]
[3]
[14]
[26]
[17]
[3]
[26]
[5]
[17]
[1]
[26]
[1]
[12]
[23]
[23]
[12]
[7]
[2]
[27, 18]
[11]
[26]
[14, 3]
[23]
[26, 13]
[23]
[26]
[17]
[26, 17]
[26]
[5]
[17]
[27]
[26]
[5]
[2]
[27]
[5]
[26, 6]
[27]
[26]
[14]
[27, 18]
[5]
[17]
[26]
[3]
[2]
[27]
[23, 20]
[26]
[17]
[14]
[17]
[26, 3]
[18]
[18]
[2]
[23, 20]
[17]
[26]
[17]
[5]
[26]
[26]
[26]
[10]
[5]
[17]
[5]
[26]
[3]
[23]
[1]
[3]
[23]
[19]
[27, 3]
[5]
[5]
[5]
[5]
[2]
[17]
[5]
[27, 17]
[26]
[14]
[14]
[15, 21]
[26]
[17, 14, 3]
[26]
[26]
[26]
[14, 5]
[23]
[1]
[27]
[2]
[13]
[3]
[1]
[1]
[27]
[26]
[10]
[1]
[5]
[26, 4]
[26]
[5]
[18]
[17]
[19]
[5]
[23]
[20]
[3]
[10]
[26, 4]
[26]
[26, 18]
[3]
[26]
[18]
[27, 6]
[5]
[26]
[2]
[27]
[2]
[14, 5]
[9]
[26]
[5, 3]
[26]
[9, 18]
[27]
[14, 3]
[15]
[26]
[

[5]
[15, 3]
[26, 5]
[5]
[17]
[26, 13]
[17]
[26]
[1]
[5]
[9]
[27]
[5]
[22]
[26]
[2]
[26]
[26]
[5]
[5]
[4, 17]
[5]
[5]
[27]
[9]
[5]
[27, 18]
[5]
[15]
[17]
[27]
[5]
[21]
[17]
[9]
[12, 2]
[26, 5]
[5]
[18]
[23]
[5]
[18]
[3]
[23]
[4, 12, 26]
[3]
[14]
[3]
[7]
[18]
[26]
[9, 14]
[17]
[26]
[15, 3]
[5]
[2]
[1]
[17]
[20]
[3]
[2, 2]
[26]
[15]
[5]
[9, 18]
[23]
[1]
[14]
[10, 2]
[26, 3]
[20]
[3]
[5]
[26]
[3]
[3]
[13]
[26, 5]
[2]
[3]
[3]
[26]
[17]
[13]
[26]
[26]
[3]
[1]
[26]
[27]
[23]
[5]
[26, 21]
[23]
[26]
[23]
[15]
[2, 15]
[27]
[26]
[27]
[12]
[26]
[17]
[23]
[27, 17]
[26]
[23]
[17]
[12]
[5]
[5]
[17]
[9]
[15]
[26]
[26]
[3]
[26]
[26]
[5]
[3]
[17]
[9]
[5]
[5]
[3]
[5]
[6]
[1]
[5, 15]
[25]
[2]
[2]
[13]
[27]
[26]
[27]
[5]
[27, 3]
[2]
[10]
[26, 17]
[3]
[3]
[5]
[26]
[17]
[23, 2]
[15]
[2]
[27]
[26]
[26]
[26]
[17]
[5]
[17]
[17]
[3]
[3]
[14, 3]
[13]
[23]
[17]
[5]
[27]
[27, 1]
[26, 3]
[26]
[23]
[3]
[27, 1]
[17]
[26]
[14]
[4]
[4, 17]
[8]
[5]
[12]
[2]
[14]
[20]
[22]
[3]
[5]
[5]
[3]
[17]
[5]
[26]
[26, 1, 3]
[26]
[12

...

[1]
[26]
[2]
[9]
[14, 5]
[14, 26]
[2]
[3]
[17]
[3]
[3]
[26]
[1, 21]
[14, 5]
[9, 3]
[20]
[3]
[2]
[27]
[26, 13]
[14]
[2]
[27]
[26]
[9]
[26]
[26, 17]
[26]
[27]
[26, 4]
[1]
[23]
[26]
[19]
[3]
[26]
[14, 5]
[26]
[17]
[3]
[26]
[13]
[17]
[10]
[26]
[17]
[2]
[26]
[13]
[27]
[27, 1]
[1]
[1]
[20]
[17]
[26]
[5]
[27]
[23, 20]
[15]
[3]
[17]
[23, 20]
[26]
[26]
[5]
[2]
[5]
[23, 20]
[26]
[18]
[14, 5]
[21]
[5]
[5]
[26]
[20]
[4, 13]
[23, 20]
[1, 26]
[3]
[17]
[13]
[13]
[15, 17]
[17]
[15]
[26]
[26, 14, 5]
[4]
[15]
[3]
[5, 21]
[9]
[9]
[3]
[26]
[5]
[1, 4, 4, 3]
[23]
[1]
[27]
[5]
[17]
[23, 20]
[8]
[14, 5]
[9]
[5]
[1]
[17]
[20]
[23]
[26]
[9]
[5, 18]
[17]
[26]
[26]
[27]
[17]
[11]
[14]
[17]
[12]
[10]
[26]
[3]
[26]
[26]
[5]
[3]
[26]
[5]
[27]
[27]
[26]
[5]
[1]
[26]
[1]
[26]
[14]
[23]
[27]
[26]
[17]
[27]
[1]
[10]
[23]
[26]
[13]
[17]
[3]
[26]
[12, 24]
[15]
[26]
[14]
[14, 5]
[5]
[2]
[5]
[12]
[10, 3]
[26, 5]
[25]
[3]
[19]
[5]
[12]
[26]
[14]
[17]
[26]
[2]
[17]
[17]
[1]
[14]
[23]
[17]
[13]
[27]
[26]
[5]
[5]
[3]
[4]
[2]
[5

[12]
[5]
[22]
[26]
[26]
[19]
[17]
[26]
[3]
[13]
[23]
[1]
[26]
[17]
[5]
[17]
[5]
[23]
[26, 5]
[27]
[20]
[23]
[23]
[5]
[5]
[13]
[13]
[15]
[11]
[3]
[17]
[5]
[3]
[26]
[23]
[26]
[23]
[5]
[11]
[1]
[5]
[27]
[26]
[27]
[17]
[3]
[5]
[5]
[2]
[3]
[3]
[10]
[17]
[15]
[3]
[19]
[2]
[3]
[26]
[6]
[14, 5]
[5]
[10]
[1]
[26]
[11]
[26]
[2]
[26]
[5]
[23]
[26, 17]
[2]
[3]
[5]
[5]
[18]
[18]
[15]
[27]
[26, 1]
[23]
[3]
[17]
[2]
[23]
[26, 21]
[26]
[1]
[23]
[17]
[12]
[23]
[2]
[14, 5]
[26]
[5]
[27, 17]
[12]
[1]
[27, 9]
[26, 4]
[2]
[17]
[26]
[25]
[5]
[2]
[26]
[1]
[17]
[17]
[1]
[15]
[1]
[26]
[3]
[27, 17]
[26]
[3]
[5]
[15]
[3]
[2]
[26]
[17]
[23]
[15]
[5]
[13]
[13]
[15]
[9]
[26]
[5]
[4]
[23]
[27]
[14]
[26]
[2]
[17]
[26, 10]
[1]
[3]
[1]
[26]
[18]
[17]
[26]
[23]
[26, 3]
[26]
[3]
[23]
[26]
[26]
[1]
[20]
[17]
[26, 13]
[5]
[14]
[3]
[11, 3]
[17]
[26]
[23]
[23]
[5]
[13]
[1]
[17]
[1]
[2]
[26]
[20]
[5]
[10]
[20]
[14]
[27, 3]
[10]
[17]
[5]
[26]
[3]
[5]
[27, 13]
[26]
[17]
[9]
[15]
[20]
[26]
[27]
[27, 1]
[26]
[3]
[26]
[1]
[5]
[5]


[5]
[17]
[26]
[5]
[26, 5]
[26]
[26]
[17]
[26]
[5]
[26, 17]
[3]
[5]
[26]
[1]
[24]
[27]
[26, 13]
[5]
[20]
[5]
[5]
[14, 5]
[26]
[26]
[5]
[26, 5]
[23]
[4]
[14]
[26, 5]
[2]
[5]
[17]
[3]
[27]
[27]
[5]
[2]
[27, 1]
[5]
[9]
[23]
[27]
[26]
[2]
[17]
[27, 1]
[27]
[26]
[1]
[5]
[26, 1]
[10]
[2]
[5]
[26]
[9, 5]
[26, 17]
[8, 27, 17]
[12]
[27]
[14]
[14]
[26]
[26]
[26]
[13]
[5]
[13]
[5]
[3]
[14]
[26]
[5]
[5]
[4, 5]
[13]
[1, 26]
[3]
[23]
[12]
[3]
[2]
[2]
[23]
[23]
[13]
[26]
[1]
[5]
[18]
[3]
[1]
[17]
[3]
[23]
[14, 5]
[5]
[26]
[2]
[26, 1]
[26]
[14]
[14, 5]
[26]
[23]
[27]
[4]
[26, 3]
[18]
[9]
[2]
[26, 6]
[25]
[27]
[17]
[26]
[17]
[13]
[23]
[26]
[23]
[13]
[26]
[5]
[3]
[17]
[17]
[5]
[1]
[2]
[26]
[18]
[2]
[13]
[19]
[5]
[19]
[26]
[3]
[10]
[17]
[25]
[9]
[3]
[14, 26]
[18]
[12]
[5]
[18]
[5]
[1]
[5]
[20]
[15, 13]
[12]
[13]
[26, 14]
[17]
[13]
[26]
[2]
[20]
[3]
[27]
[26]
[10]
[26]
[23]
[17]
[12]
[24]
[26]
[24]
[21]
[26]
[26]
[26]
[23]
[24]
[2]
[23]
[17]
[26]
[26]
[7]
[5]
[5]
[2]
[18]
[5]
[27, 5]
[4]
[5]
[14]
[15]
[1]


[5, 18]
[26]
[2]
[17]
[5]
[1]
[9, 18]
[26]
[2]
[27]
[20]
[9]
[26]
[14]
[5]
[15]
[5]
[26]
[9]
[5]
[27, 17]
[26]
[27]
[23]
[5]
[19]
[13]
[2]
[23]
[26]
[26]
[23, 20]
[17]
[12]
[15]
[9]
[27]
[5]
[1]
[14]
[5]
[23, 5]
[2]
[26, 5]
[1]
[23]
[19]
[2]
[26]
[14, 5]
[12]
[1]
[12]
[13]
[26]
[26]
[26]
[17]
[23]
[26]
[27, 3]
[4]
[5]
[26, 3]
[19]
[6]
[11]
[23]
[27]
[14, 15]
[5]
[26, 1]
[17]
[4, 26, 17]
[26]
[19, 5]
[17]
[3]
[26]
[26, 5]
[2]
[27]
[13]
[27, 1]
[5]
[1]
[5]
[26, 3]
[5]
[26]
[4, 5]
[13]
[26]
[5]
[23]
[26, 5]
[26]
[5]
[26, 14]
[26, 17]
[4]
[17]
[17]
[2]
[5]
[2]
[5]
[6]
[5]
[20]
[1]
[26]
[5]
[14]
[2]
[26]
[14]
[3]
[5]
[15]
[5]
[3]
[26, 5]
[2]
[26]
[5]
[20]
[15]
[13]
[26]
[17]
[5]
[5]
[17]
[10]
[26]
[5]
[27]
[5]
[1]
[23]
[23]
[26]
[5]
[26, 3]
[17]
[21]
[10]
[26]
[26, 18]
[17]
[26, 13]
[26]
[5]
[17]
[5]
[1]
[26]
[23]
[15]
[23]
[17]
[17]
[17]
[18]
[26]
[20]
[27, 1]
[2]
[27]
[27]
[6]
[5]
[12]
[5]
[23]
[2]
[26]
[2]
[17]
[13]
[5]
[13]
[26]
[10]
[23, 5]
[26]
[26, 3]
[12]
[3]
[27]
[26]
[2]
[18]
[17]

[26, 14, 5]
[26]
[17]
[13]
[3]
[5]
[26]
[5]
[2]
[27]
[15]
[5]
[9]
[26]
[17]
[17]
[15]
[17]
[26]
[26]
[17]
[2]
[1]
[26]
[17]
[26]
[17]
[26]
[17]
[2]
[26]
[9, 4]
[17]
[5]
[17]
[2]
[27, 1]
[15, 3]
[5]
[3]
[23]
[17]
[26]
[3]
[15]
[12]
[2]
[13]
[1]
[18]
[26]
[17]
[3]
[10]
[17]
[10]
[5, 26]
[27]
[16]
[1]
[3]
[3]
[5]
[5]
[23]
[26]
[23]
[23]
[15]
[27, 17]
[2]
[13]
[26]
[23]
[26]
[10]
[4]
[26]
[26]
[17]
[5]
[26]
[1]
[3]
[10]
[5]
[5]
[9]
[26]
[26]
[18]
[26]
[23]
[27]
[27]
[14]
[13]
[9, 17]
[23]
[26]
[17]
[18]
[26]
[27]
[14, 5]
[13]
[27]
[5]
[10]
[5]
[5]
[27, 1]
[2]
[26]
[19]
[26]
[10]
[26]
[23, 20]
[5]
[13]
[23]
[26]
[13]
[1]
[2]
[23]
[5]
[27]
[26]
[5]
[1]
[5]
[10, 12, 10]
[17]
[5]
[26]
[9]
[18]
[1, 26]
[26]
[23]
[9, 1]
[26]
[9]
[26, 13]
[3]
[3]
[3]
[5]
[23]
[26]
[3]
[5]
[1]
[17]
[23, 12]
[1]
[26]
[2]
[5]
[23]
[5]
[26]
[5]
[17]
[17]
[13]
[14]
[20]
[18]
[9, 5]
[26]
[19]
[5]
[26, 5]
[26]
[14]
[5]
[26]
[9]
[17]
[20]
[9]
[14, 17]
[17]
[23]
[10]
[9]
[24]
[5]
[5]
[14]
[3]
[17]
[26, 21]
[26]
[26]
[5]
[

In [61]:
# Shows that the correct result is obtained
df.iloc[4]

Ingredients           [ground beef, tomato, water, green bean, carro...
Recipe_id                                              Tomato Beef Stew
Ingredient_Numeric                 [817, 12333, 430, 989, 20786, 20132]
Recipe_id_numeric                                                     1
Name: 4, dtype: object

In [62]:
df.Recipe_id_numeric.value_counts()

0     63952
26     6104
5      5932
3      3366
17     3337
23     2138
2      2007
1      1959
13     1476
27     1321
14     1189
15      996
18      964
12      812
10      788
9       774
4       630
20      588
24      399
19      273
6       223
21      201
11      193
25      113
7        94
8        76
22       55
16       40
Name: Recipe_id_numeric, dtype: int64

In [63]:
len(df)

100000

## I am pickling both the vocabularies which would be useful later

In [64]:
with open('vocab_combined.pkl','wb') as file:
    pickle.dump(recipe_vocabulary, file)
    pickle.dump(ingredient_vocabulary, file)

## Let us drop the empty entries

In [65]:
def check_empty(x):
    return len(x) == 0
    

In [66]:
processed_df = df.drop(df[df.Ingredient_Numeric.apply(check_empty)].index)

In [67]:
assert sum(processed_df.Ingredient_Numeric.apply(check_empty)) == 0

In [68]:
# I am just saving the file here for reference later
pd.to_pickle(processed_df, './processed_df_glove.pkl')

## Let us build the two set of data using sklearn dataloaders 

In [44]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(processed_df, test_size = 0.2, random_state = 0)

# This is the Neural Network portion

In [45]:
# I am assuming that we are able to get the categories here
# Now this should become a LSTM based model which will try and do binary prediction
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.utils.data.dataloader as dataloader
from torch.nn.utils.rnn import pad_sequence

In [46]:
# Some constants
batch_size = 5

In [47]:
# https://github.com/yunjey/seq2seq-dataloader/blob/master/data_loader.py
def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (src_seq, trg_seq).
    We should build a custom collate_fn rather than using default collate_fn,
    because merging sequences (including padding) is not supported in default.
    Seqeuences are padded to the maximum length of mini-batch sequences (dynamic padding).
    Args:
        data: list of tuple (src_seq, trg_seq).
            - src_seq: torch tensor of shape (?); variable length.
            - trg_seq: torch tensor of shape (?); variable length.
    Returns:
        src_seqs: torch tensor of shape (batch_size, padded_length).
        src_lengths: list of length (batch_size); valid length for each padded source sequence.
        trg_seqs: torch tensor of shape (batch_size, padded_length).
        trg_lengths: list of length (batch_size); valid length for each padded target sequence.
    """
    def merge(sequences):
        lengths = [len(seq) for seq in sequences]
        padded_seqs = torch.zeros(len(sequences), max(lengths)).long()
        for i, seq in enumerate(sequences):
            end = lengths[i]
            padded_seqs[i, :end] = seq[:end]
        return padded_seqs, lengths

    # sort a list by sequence length (descending order) to use pack_padded_sequence
#     print(data[0]) # list of tuples
    data.sort(key=lambda x: len(x[0]), reverse=True)

    # seperate source and target sequences
    src_seqs, trg_seqs = zip(*data)

    # merge sequences (from tuple of 1D tensor to 2D tensor)
    src_seqs, src_lengths = merge(src_seqs)
    # target sequence for us is a single tensor so we do not need to 
    # merge it
    #trg_seqs, trg_lengths = merge(trg_seqs)
    trg_seqs = torch.as_tensor(trg_seqs)
    return src_seqs, src_lengths, trg_seqs #, trg_lengths


In [48]:
class RecipeData(data.Dataset):
    
    def __init__(self, df):
        super(RecipeData, self).__init__()
        self.df = df
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        X = torch.as_tensor(self.df.Ingredient_Numeric.iloc[idx])
        y = torch.as_tensor(self.df.Recipe_id_numeric.iloc[idx])
        return X,y
    

In [49]:
train_dataset = RecipeData(df_train)
test_dataset = RecipeData(df_test)
train_data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                              batch_size=batch_size,
                                              shuffle=True,
                                              collate_fn=collate_fn,
                                               drop_last=True) # Done for cases when num_samples not exact multiple
test_data_loader = torch.utils.data.DataLoader(dataset=test_dataset, # of the batch_size
                                              batch_size=batch_size,
                                              shuffle=True,
                                              collate_fn=collate_fn,
                                              drop_last=True)

In [50]:
df.Recipe_id_numeric[4]

5

In [51]:
train_dataset[0]

(tensor([ 65,  33, 140,  36]), tensor(0))

In [52]:
for (X,X_len,y) in train_data_loader:
    print(X)
    print(X_len)
    print(y)
    break

tensor([[ 19,   3, 174,  39,  95,  30,  52,  18, 225,   8,  38],
        [130, 281,  38, 721,  24,  69, 129, 287, 278, 131,   0],
        [486, 331,  23, 187,  80, 401, 185, 186, 341,   0,   0],
        [131, 167, 151, 263,  61, 292,  52,  30,   0,   0,   0],
        [129,   3,  94, 137, 196,   0,   0,   0,   0,   0,   0]])
[11, 10, 9, 8, 5]
tensor([1, 0, 0, 0, 0])


In [53]:
X

tensor([[ 19,   3, 174,  39,  95,  30,  52,  18, 225,   8,  38],
        [130, 281,  38, 721,  24,  69, 129, 287, 278, 131,   0],
        [486, 331,  23, 187,  80, 401, 185, 186, 341,   0,   0],
        [131, 167, 151, 263,  61, 292,  52,  30,   0,   0,   0],
        [129,   3,  94, 137, 196,   0,   0,   0,   0,   0,   0]])

In [54]:
class RecipePredictor(nn.Module):
    
    def __init__(self, vocab_size, hidden_dim, embedding_dim, batch_size, output_dim):
        super(RecipePredictor, self).__init__()
        self.vocab_size = vocab_size
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.lstm = nn.LSTM(input_size=self.embedding_dim, hidden_size=self.hidden_dim)
        self.batch_size = batch_size
        self.predictor = nn.Linear(self.hidden_dim, self.output_dim)
        self.non_linearity = nn.ReLU()
        self.init_hidden() # TODO:  This should happen at the beginning of each epoch
        
    def init_hidden(self):
        self.h_n = torch.randn(1, self.batch_size, self.hidden_dim)
        self.c_n = torch.randn(1, self.batch_size, self.hidden_dim)
        
    
    def forward(self, input_sequence, max_len):
#         print(input_sequence)
#         print(max_len)
        embedded = self.embedding(input_sequence)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, max_len, batch_first=True)
        outputs, (self.c_n, self.h_n) = self.lstm(packed, (self.c_n, self.h_n))
        # Unpack padding
        """
            Honestly, I do not know if at this point, I need the output. I would rather prefer to work with the
            self.h_n cell and so will not `pad_padded_sequence`
        """
        #outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        # on which output should the prediction be done?
        # self.h_n =  num_layers, batch_size, hidden_dim
        batch_size = self.h_n.shape[1]
        output_predicted = self.predictor(self.h_n.reshape(batch_size, -1))
        return output_predicted
        

In [55]:
vocab_size = ingredient_vocabulary.max_idx + 1

In [56]:
num_recipes = recipe_vocabulary.max_idx+1
model = RecipePredictor(vocab_size=vocab_size, hidden_dim=512, embedding_dim=300, batch_size=batch_size,
                       output_dim=num_recipes)

In [57]:
model(X, X_len)

tensor([[ 0.0401,  0.2132,  0.0743, -0.1556,  0.0510,  0.1881, -0.0421,  0.1119,
          0.0713,  0.1193,  0.0347, -0.0798, -0.1863, -0.0639, -0.1612, -0.0156],
        [ 0.0567,  0.0471,  0.0292,  0.0956, -0.0449, -0.0879, -0.0174, -0.1688,
          0.1125, -0.1681,  0.0670, -0.1165,  0.0689,  0.0322, -0.0800,  0.0156],
        [-0.0313,  0.0320, -0.0326,  0.1923,  0.1740, -0.0895, -0.0408,  0.0142,
          0.0165, -0.0118,  0.0636, -0.0233, -0.2884,  0.2130,  0.0235,  0.1105],
        [ 0.1292, -0.0452,  0.1735,  0.2075, -0.1769,  0.0785,  0.1529, -0.1209,
          0.1062,  0.1867,  0.0799, -0.0008,  0.0811, -0.1242,  0.0404, -0.1528],
        [ 0.0271, -0.1232,  0.3260,  0.1704,  0.0706,  0.0453,  0.0249,  0.1153,
          0.2356, -0.0587,  0.2022, -0.1504, -0.0504, -0.0279, -0.0954, -0.0797]],
       grad_fn=<AddmmBackward>)

# Put loss function, gradient and try to train the lstm

In [58]:
lr = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [59]:
num_epoch = 2

In [60]:
# Moving things to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [61]:
model.to(device)

RecipePredictor(
  (embedding): Embedding(1153, 300)
  (lstm): LSTM(300, 512)
  (predictor): Linear(in_features=512, out_features=16, bias=True)
  (non_linearity): ReLU()
)

In [62]:
def put_elements_to_device(a,b,c,device):
    return a.to(device), b.to(device), c.to(device)

In [1]:
# !pip install -q tb-nightly

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [74]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [75]:
print(len(train_data_loader) * batch_size)
print(len(df_train))
# Not same since we are dropping some terms which do not match up
N_train = len(train_data_loader) * batch_size
N_test = len(test_data_loader) * batch_size

79945
79948


In [76]:
for epoch in range(num_epoch):
    running_loss = 0
    model.init_hidden()
    for idx,(X,X_len,y) in enumerate(train_data_loader):
        model.zero_grad()
        X, X_len, y = put_elements_to_device(a=X, b=torch.tensor(X_len), c=y, device=device)
        prediction = model(X, X_len)
        loss = criterion(prediction, y)
        loss.backward(retain_graph=True)
        optimizer.step()
        running_loss += loss.item()
        if idx % 100 == 0:
            print("epoch {} loss {}".format(epoch, running_loss))
    writer.add_scalar('Loss/train', running_loss/N_train, epoch)
    # validation set is still left to create
    # Now to test the validation set
    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for idx,(X,X_len,y) in enumerate(test_data_loader):
            X, X_len, y = put_elements_to_device(a=X, b=torch.tensor(X_len), c=y, device=device)
            outputs = model(X,X_len)
            loss = criterion(outputs, y)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += X.size(0)
            correct += (predicted == y).sum().item()
    writer.add_scalar('Loss/train', test_loss/N_test, epoch)
    print('Accuracy of the network on the test samples: %d %%' % (
        100 * correct / total))
        
            

epoch 0 loss 2.0085887908935547
epoch 0 loss 102.51984167844057
epoch 0 loss 221.88543963432312
epoch 0 loss 312.94191985577345
epoch 0 loss 409.68190402537584


KeyboardInterrupt: 

In [67]:
%load_ext tensorboard
%tensorboard --logdir=runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [78]:
torch.__version__

'1.3.0'

In [80]:
vindhya_data_df = pd.read_csv('./recipe_title_foodCategory.csv')

In [83]:
list(set(vindhya_data_df.Food_Category))

['Cake',
 'Salad',
 'Noodles',
 'Hummus',
 'Burger',
 'Steak',
 'Pie',
 'Chicken',
 'Custard',
 'Chips',
 'Casserole',
 'Pasta',
 'Pizza',
 'Omelette']

In [94]:
a = [1,2,3,4,5]
b = ['a','b','c','d','e']
c = ['dil','me','dard','e','disco']
sallu_bhoi = zip(a,b,c)

In [95]:
a,b,c = zip(*sallu_bhoi)

In [100]:
np.asarray(a)

array([1, 2, 3, 4, 5])